# Pipeline training

This notebook is used for triggering the training pipeline on the basis of the steps described in the notebook mlops_dnn.ipynb

In [ ]:
import boto3
from datetime import datetime
import json
import logging
import os
import pandas as pd
from sagemaker import get_execution_role, image_uris
from sagemaker.tensorflow import TensorFlow
import sys
import time
import traceback
import yaml

In [ ]:
sys.path.insert(0, os.path.abspath('./../mlpipelines/pipelines'))

In [ ]:
from inference.pipeline import get_pipeline

***

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Global parameters

In [ ]:
region = boto3.session.Session().region_name
role = get_execution_role()
deployment_configs = {
    "ComponentName": "mlops-iot-regressor",
    "PlatformOS": "LINUX",
    "PlatformArchitecture": "X86_64"
}
lambdas = {
    "get-last-approved-model": "",
    "create-packaging-job": "",
    "check-packaging-job": ""
}
s3_bucket_name = "isengard-bpistone-mlops-iot-dev"

***

## Inference pipeline

### Get pipeline definition

In [ ]:
pipeline = get_pipeline(
    region,
    deployment_configs,
    lambdas,
    s3_bucket_name
)

### Create or update SageMaker pipeline

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
json.loads(pipeline.definition())

### Pipeline parameters

In [ ]:
edge_model_name = "mlops-iot-regressor"
model_package_group_name = "mlops-iot-regressor"

### Start training pipeline 

In [ ]:
execution = pipeline.start(
    parameters=dict(
        edge_model_name=edge_model_name,
        model_package_group_name=model_package_group_name
    )
)

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()